### Preamble

Loading necessary libraries.

In [2]:
# Configure libraries

%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
import geohash
from sklearn.metrics import mean_squared_error
import random
import math
import datetime

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=2, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_
    print "Best score: ", gs.best_score_
    best = gs.best_estimator_
    return best

###  Reading the taxi data

Read in the training and validation data. Training data is from January 2013 till February 2015. Validation data is from March and April 2015. This data was prepared on an Amazon EC2 Spark cluster, transferred to S3 and then downloaded. It was converted from an RDD to a csv using the [ConvertRDDtoCSV](https://github.com/sdaulton/TaxiPrediction/blob/master/3. ConvertRDDtoCSV.ipynb) notebook.

In [4]:
# Each line is of the format:
# ((time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend, geohash), number of pickups)
names = ["year","month","day","time_cat", "time_num", "time_cos", "time_sin", "day_cat", "day_num", "day_cos", "day_sin", "weekend", "geohash", "pickups"]
dftrain=pd.read_csv("./data/trainset7.csv", header=None, names = names)
print dftrain.shape

(695424, 14)


In [5]:
dfvalid=pd.read_csv("./data/validset7.csv", header=None, names = names)
print dfvalid.shape

(37305, 14)


### Reading and preparing the weather data

Here we are reading the weather data pulled from NOAA for New York City Central Park. It contains among other things, daily numbers of precipitation, snowfall, snow depth, wind speed and minimum and maximum temperatures.

In [6]:
# Get the weather data

weather=pd.read_csv("./data/nyc-weather-data.csv")
weather.ix[weather.SNWD <= -9999, 'SNWD'] = 0
weather.ix[weather.SNOW <= -9999, 'SNOW'] = 0
weather.ix[weather.AWND <= -9999, 'AWND'] = 0
weather['year'] = (weather['DATE']/10000).apply(math.floor)
weather['month'] = ((weather['DATE'].mod(10000))/100).apply(math.floor)
weather['day'] = weather['DATE'].mod(100)
weather = weather[['year','month','day','PRCP','SNWD','SNOW','TMAX','TMIN','AWND']]
weather['PRCP'] = weather['PRCP'] / 10.
weather['TMAX'] = weather['TMAX'] / 10.
weather['TMIN'] = weather['TMIN'] / 10.
weather['AWND'] = weather['AWND'] / 10. * 3.6
weather.columns = ['year','month','day','precipitation','snow_depth','snowfall','max_temp','min_temp','avg_wind']
weather.head()

,year,month,day,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind
0,2009,1,1,0,0,0,-3.3,-9.4,18.00
1,2009,1,2,0,0,0,1.1,-5.0,10.08
2,2009,1,3,0,0,0,3.3,-1.7,16.20
3,2009,1,4,0,0,0,5.6,-3.9,12.24
4,2009,1,5,0,0,0,6.1,3.3,11.16


### Final preperation for machine learning

I tried initially doing one-hot-encoding on the geohashes to generate numerical features, but I quickly realized that this was not feasible from a memory perspective. Millions of records times tens of thousands of features would not fit in memory. So I decided to go for the numerical latitude and longitude route. Using a random forest, we can easily detect higher order structures in these two variables.

First we define some functions to further prepare the data.

In [7]:
# Get the number of the day in the year. So January 1 is 1, December 31st is 365 or 366
def get_yearday(df):
    date = datetime.date(df['year'],df['month'],df['day'])
    return (date.timetuple().tm_yday-1)/365.

# Get the longitude and latitude from the geohash
def decodegeo(geo, which):
    if len(geo) >= 6:
        geodecoded = geohash.decode(geo)
        return geodecoded[which]
    else:
        return 0

# Join the weather data to the taxi data, add some more time and date features and
# get the latitude and longitude of the center of the geohash where the records were bucketed in.
def further_data_prep(df):
    df = pd.merge(df, weather, how='left', on=['year','month','day'])
    df['year_num'] = df.apply(lambda x:get_yearday(x),axis=1)
    df['month_num'] = (df['day']-1)/30.
    df['year_sin'] = (df['year_num'] * 2 * math.pi).apply(math.sin)
    df['year_cos'] = (df['year_num'] * 2 * math.pi).apply(math.cos)
    df['month_sin'] = (df['month_num'] * 2 * math.pi).apply(math.sin)
    df['month_cos'] = (df['month_num'] * 2 * math.pi).apply(math.cos)
    df['latitude'] = df['geohash'].apply(lambda geo: decodegeo(geo, 0))
    df['longitude'] = df['geohash'].apply(lambda geo: decodegeo(geo, 1))
    return df

Prepare the data.

In [8]:
%%time
dftrain = further_data_prep(dftrain)
dfvalid = further_data_prep(dfvalid)

CPU times: user 32.3 s, sys: 1.22 s, total: 33.5 s
Wall time: 34.4 s


In [9]:
# Check results
dfvalid.head()

,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,geohash,pickups,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind,year_num,month_num,year_sin,year_cos,month_sin,month_cos,latitude,longitude
0,2013,9,25,14:30,0.614583,-0.75184,-0.659346,Wednesday,0.373512,-0.700465,0.713687,0,dr5rybc,1,0,0,0,22.8,11.1,2.88,0.731507,0.800000,-0.993257,-0.115935,-0.951057,0.309017,40.742111,-73.880997
1,2013,9,25,21:00,0.885417,0.75184,-0.659346,Wednesday,0.412202,-0.851662,0.524092,0,dr72j1s,1,0,0,0,22.8,11.1,2.88,0.731507,0.800000,-0.993257,-0.115935,-0.951057,0.309017,40.790176,-73.953781
2,2013,9,28,22:30,0.947917,0.94693,-0.321439,Saturday,0.849702,0.586271,-0.810115,1,dr72j38,1,0,0,0,22.8,13.3,7.20,0.739726,0.900000,-0.997917,-0.064508,-0.587785,0.809017,40.790176,-73.948288
3,2013,9,9,09:00,0.385417,-0.75184,0.659346,Monday,0.055060,0.940754,0.339090,0,dr5rxth,1,0,0,0,22.8,13.9,6.84,0.687671,0.266667,-0.924291,-0.381689,0.994522,-0.104528,40.721512,-73.843918
4,2013,9,18,02:00,0.093750,0.83147,0.555570,Wednesday,0.299107,-0.303677,0.952775,0,dr5rkqc,1,0,0,0,22.2,10.6,4.32,0.712329,0.566667,-0.972118,-0.234491,-0.406737,-0.913545,40.687180,-73.990860


In [10]:
print dftrain.count()
print dfvalid.count()

year             695424
month            695424
day              695424
time_cat         695424
time_num         695424
time_cos         695424
time_sin         695424
day_cat          695424
day_num          695424
day_cos          695424
day_sin          695424
weekend          695424
geohash          695424
pickups          695424
precipitation    695424
snow_depth       695424
snowfall         695424
max_temp         695424
min_temp         695424
avg_wind         695424
year_num         695424
month_num        695424
year_sin         695424
year_cos         695424
month_sin        695424
month_cos        695424
latitude         695424
longitude        695424
dtype: int64
year             37305
month            37305
day              37305
time_cat         37305
time_num         37305
time_cos         37305
time_sin         37305
day_cat          37305
day_num          37305
day_cos          37305
day_sin          37305
weekend          37305
geohash          37305
pickups         

Extract the features that will actually be used in the machine learning algorithm and define the target. The target in this case will be the logarithm of the number of pickups. Why the logarithm? Because if we would use the number of pickups as is, mistakes in absolute numbers would be treated equally among high traffic locations and low traffic locations. An error of 100 pickups in a location with usually about 1000 pickups would be treated equally as an error of 100 pickups in a location with usually about 10 pickups. That is not what we want. We also want some accuracy in the low traffic locations. Taking the logarithm allows us to do that. With a logarithmic target, errors are optimized in an order of magnitude scale, rather than an absolute scale. So the result will be that predictions are correct to such and such order of magnitude.

In [11]:
# Split off the features
# Xnames = ['month','day','latitude','longitude','year_num','year_cos','year_sin',
#           'month_num','month_cos','month_sin','day_num','day_cos','day_sin',
#           'time_num','time_cos','time_sin','weekend','precipitation',
#           'snow_depth','snowfall','max_temp','min_temp','avg_wind']
Xnames = ['month','day','latitude','longitude','year_num','year_cos','year_sin',
          'month_cos','month_sin','day_num','day_cos','day_sin',
          'time_num','time_cos','time_sin','max_temp','min_temp','avg_wind']
Xtrain = dftrain[Xnames]
Xvalid = dfvalid[Xnames]

# Split off the target (which will be the logarithm of the number of pickups (+1))
ytrain = np.log10(dftrain['pickups']+1)
yvalid = np.log10(dfvalid['pickups']+1)

### Get the training and test sets

### Random Forest Regression

The best algorithm that we tried on this problem was a Random Forest regressor. Using deep trees, a random forest is able to really get at the detailed structure of the longitude and latitude features to really use specific regions of NYC to make its predictions. We're evaluating the random forest on root-mean-squared-error (RMSE) of the prediction (in log-space of number of pickups).

In [12]:
# Create a Random Forest Regression estimator
reg = RandomForestRegressor(n_estimators=5, n_jobs=-1)

When we did our first tests with this dataset, we used the `cv_optimize` function (see the cell below) to try out several configurations of parameters using cross validation to get a feeling for the influence on accuracy (model quality) and performance (runtime). We concluded a few things from this:

1. It quickly became apparent that more than about 2 million records lead to very long runtimes.
2. Trees deeper than 20 were not adding much quality.
3. The more trees we created, the more accurate the model, but also the longer the runtime.
4. Using all features ("auto") was better than using only sqrt(n) features or log2(n) features

For this version of the notebook, we went with 50 trees of depth 20, using a sample of 2,000,000 data points.

The code below can be used to train the model using 50 trees of depth 20. After training each tree, the RMSE on the training and validation sets is calculated to visualize the progress we're making in model quality.

In [13]:
reg = RandomForestRegressor(n_estimators=1, max_depth=5, n_jobs=-1, warm_start=True)
for n in range(1,2):
    reg.set_params(n_estimators=n)
    reg.fit(Xtrain,ytrain)
    training_accuracy = reg.score(Xtrain, ytrain)
    valid_accuracy = reg.score(Xvalid, yvalid)
    rmsetrain = np.sqrt(mean_squared_error(reg.predict(Xtrain),ytrain))
    rmsevalid = np.sqrt(mean_squared_error(reg.predict(Xvalid),yvalid))
    print "N = %d, R^2 (train) = %0.3f, R^2 (valid) = %0.3f, RMSE (train) = %0.3f, RMSE (valid) = %0.3f" % (n,training_accuracy, valid_accuracy, rmsetrain, rmsevalid)

N = 1, R^2 (train) = 0.146, R^2 (valid) = -0.067, RMSE (train) = 0.115, RMSE (valid) = 0.094


### Evaluate the results

Sanity check on some records.

In [14]:
# Show some of the predictions vs. the real number of pickups
# predictions vs. real number of pickups
pd.DataFrame(np.round(np.power(10,np.column_stack((reg.predict(Xvalid),yvalid))) - 1,decimals=0).astype(int)).head(20)

,0,1
0,1,1
1,1,1
2,1,1
3,5,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [15]:
# Calculate the Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(reg.predict(Xvalid),yvalid))
print "RMSE = %0.3f (this is in log-space!)" % rmse
print "So two thirds of the records would be a factor of less than %0.2f away from the real value." % np.power(10,rmse)

RMSE = 0.094 (this is in log-space!)
So two thirds of the records would be a factor of less than 1.24 away from the real value.


In [16]:
# What are the most important features?
import operator
dict_feat_imp = dict(zip(list(Xtrain.columns.values),reg.feature_importances_))
sorted_features = sorted(dict_feat_imp.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

[('longitude', 0.46359825781232655),
 ('latitude', 0.4557488842520679),
 ('year_num', 0.03944925259961964),
 ('year_sin', 0.031636174588914275),
 ('year_cos', 0.006439938190566821),
 ('time_sin', 0.003127492556504644),
 ('avg_wind', 0.0),
 ('time_num', 0.0),
 ('day_num', 0.0),
 ('month_sin', 0.0),
 ('day_cos', 0.0),
 ('month_cos', 0.0),
 ('min_temp', 0.0),
 ('time_cos', 0.0),
 ('month', 0.0),
 ('day', 0.0),
 ('day_sin', 0.0),
 ('max_temp', 0.0)]

Latitude and longitude are obviously the most important factors influencing the number of pickups. After that come time of the day and day of the week (in several incarnations). The average wind speed is the most important weather factor, corresponding nicely to a [2014 Taxicab Fact book](http://www.nyc.gov/html/tlc/downloads/pdf/2014_taxicab_fact_book.pdf) from TLC stating that a blizzard, a hurricane and a superstorm were the most important weather events influencing taxi rides. After wind speed comes the first seasonal variable.

### Create predictions for visualization

Now we are going to generate predictions that we can visualize in Tableau. We do this by generating all possible combinations of time and location so that we have a well filled space of predictions. Then we generate predictions for all these combinations and then export to .csv.

#### Load the test set

In [17]:
dftest=pd.read_csv("./data/testset7.csv", header=None, names = names)
print dftest.shape

(410954, 14)


Prepare the data in the same way as the training and validation sets.

In [18]:
dftest = dftest.sample(1000)

In [19]:
%%time
dftest = further_data_prep(dftest)

CPU times: user 68 ms, sys: 12 ms, total: 80 ms
Wall time: 103 ms


In [20]:
dftest.head()

,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,geohash,pickups,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind,year_num,month_num,year_sin,year_cos,month_sin,month_cos,latitude,longitude
0,2013,11,17,05:00,0.218750,0.195090,0.980785,Sunday,0.888393,0.764037,-0.645172,1,dr5rtmn,1,0.8,0,0,15.6,10.6,5.76,0.876712,0.533333,-0.699458,0.714673,-2.079117e-01,-0.978148,40.721512,-73.940048
1,2013,11,26,19:30,0.822917,0.442289,-0.896873,Tuesday,0.260417,-0.065403,0.997859,0,dr5rxpn,2,13.0,0,0,8.3,0.6,6.84,0.901370,0.833333,-0.580800,0.814046,-8.660254e-01,0.500000,40.732498,-73.863144
2,2013,11,27,16:00,0.677083,-0.442289,-0.896873,Wednesday,0.382440,-0.739379,0.673289,0,dr72jv5,1,50.3,0,0,16.7,1.7,15.48,0.904110,0.866667,-0.566702,0.823923,-7.431448e-01,0.669131,40.809402,-73.922195
3,2013,11,13,19:00,0.802083,0.321439,-0.946930,Wednesday,0.400298,-0.810115,0.586271,0,dr5rv8d,1,0.0,0,0,3.9,-1.6,9.72,0.865753,0.400000,-0.746972,0.664855,5.877853e-01,-0.809017,40.740738,-73.934555
4,2013,10,31,18:00,0.760417,0.065403,-0.997859,Thursday,0.537202,-0.972805,-0.231627,0,dr72mst,1,1.5,0,0,18.9,11.7,5.76,0.830137,1.000000,-0.875892,0.482508,-2.449294e-16,1.000000,40.850601,-73.930435


### Create an artificial dataset
This dataset will contain all possible combinations of time and location to predict for on May 1st, 2015, the day we want to visualize.

In [35]:
# Get all the different times with its corresponding weather data on May 1st, 2015

# Get the May 1, 2015 data
time_data = dftest[(dftest['month'] == 11) & (dftest['day'] == 13)]

# Drop the features we don't need and drop duplicates to get unique values.
time_data = time_data.drop(['geohash','pickups','year','time_cat','day_cat','latitude','longitude'], axis=1).drop_duplicates()
print time_data.shape
time_data.head()

(10, 21)


,month,day,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind,year_num,month_num,year_sin,year_cos,month_sin,month_cos
3,11,13,0.802083,0.321439,-0.946930,0.400298,-0.810115,0.586271,0,0,0,0,3.9,-1.6,9.72,0.865753,0.4,-0.746972,0.664855,0.587785,-0.809017
7,11,13,0.510417,-0.997859,-0.065403,0.358631,-0.630773,0.775968,0,0,0,0,3.9,-1.6,9.72,0.865753,0.4,-0.746972,0.664855,0.587785,-0.809017
9,11,13,0.260417,-0.065403,0.997859,0.322917,-0.442289,0.896873,0,0,0,0,3.9,-1.6,9.72,0.865753,0.4,-0.746972,0.664855,0.587785,-0.809017
102,11,13,0.281250,-0.195090,0.980785,0.325893,-0.458982,0.888446,0,0,0,0,3.9,-1.6,9.72,0.865753,0.4,-0.746972,0.664855,0.587785,-0.809017
314,11,13,0.343750,-0.555570,0.831470,0.334821,-0.508075,0.861313,0,0,0,0,3.9,-1.6,9.72,0.865753,0.4,-0.746972,0.664855,0.587785,-0.809017


In [36]:
# Get all the different locations

# First figure out which columns to drop
time_cols = list(Xtrain.columns.values)
time_cols.remove('latitude')
time_cols.remove('longitude')

# Then get the unique locations
loc_data = Xtrain.drop(time_cols, axis=1).drop_duplicates()

# To reduce memory consumption in Tableau, we are only predicting for
# the region closely around Manhattan and the La Guardia and JFK airports
loc_data = loc_data[(loc_data['latitude'] > 40.5) & (loc_data['latitude'] < 41.1) &
                    (loc_data['longitude'] > -74.1) & (loc_data['longitude'] < -73.6)]

print loc_data.shape
loc_data.head()

(14203, 2)


,latitude,longitude
0,40.806656,-73.953781
1,40.758591,-73.937302
2,40.831375,-73.857651
3,40.797043,-73.938675
4,40.832748,-73.861771


In [37]:
# Dummy column to be able to join the times with the locations
time_data['key'] = 1
loc_data['key'] = 1

print time_data.shape
print loc_data.shape

(10, 22)
(14203, 3)


In [38]:
# Merge the time_data and location_data
result = pd.merge(time_data, loc_data, on='key').drop(['key'], axis=1)


In [39]:
result.shape

(142030, 23)

In [40]:
#result = result[Xtrain.columns.values]
print result.shape[0]
result = result[Xnames]
result.head()

142030


,month,day,latitude,longitude,year_num,year_cos,year_sin,month_cos,month_sin,day_num,day_cos,day_sin,time_num,time_cos,time_sin,max_temp,min_temp,avg_wind
0,11,13,40.806656,-73.953781,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72
1,11,13,40.758591,-73.937302,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72
2,11,13,40.831375,-73.857651,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72
3,11,13,40.797043,-73.938675,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72
4,11,13,40.832748,-73.861771,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72


#### Then we do the prediction

In [41]:
# Get the real number of pickups and take care that we can merge it with the predictions,
yy = dftest[['month','day','day_num','latitude','longitude','pickups']]
yy = yy[(yy['month'] == 5) & (yy['day'] == 1)]

# Do predictions and convert the logarithm to the normal numbers
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1

# Merge the predictions and the real pickups
result = pd.merge(result, yy, how='left', on=['month','day','day_num','latitude','longitude'])
result.head(10)

,month,day,latitude,longitude,year_num,year_cos,year_sin,month_cos,month_sin,day_num,day_cos,day_sin,time_num,time_cos,time_sin,max_temp,min_temp,avg_wind,pred_pickups,pickups
0,11,13,40.806656,-73.953781,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.629808,NaN
1,11,13,40.758591,-73.937302,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.201256,NaN
2,11,13,40.831375,-73.857651,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.041130,NaN
3,11,13,40.797043,-73.938675,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.201256,NaN
4,11,13,40.832748,-73.861771,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.041130,NaN
5,11,13,40.688553,-73.988113,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.054482,NaN
6,11,13,40.753098,-73.913956,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.201256,NaN
7,11,13,40.803909,-73.955154,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.629808,NaN
8,11,13,40.798416,-73.952408,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.201256,NaN
9,11,13,40.840988,-73.940048,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.649880,NaN


In [42]:
print result.shape[0]
result.head()

142030


,month,day,latitude,longitude,year_num,year_cos,year_sin,month_cos,month_sin,day_num,day_cos,day_sin,time_num,time_cos,time_sin,max_temp,min_temp,avg_wind,pred_pickups,pickups
0,11,13,40.806656,-73.953781,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.629808,NaN
1,11,13,40.758591,-73.937302,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.201256,NaN
2,11,13,40.831375,-73.857651,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.041130,NaN
3,11,13,40.797043,-73.938675,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.201256,NaN
4,11,13,40.832748,-73.861771,0.865753,0.664855,-0.746972,-0.809017,0.587785,0.400298,-0.810115,0.586271,0.802083,0.321439,-0.94693,3.9,-1.6,9.72,1.041130,NaN


In [43]:
# Drop unnecessary columns to reduce memory consumption in Tableau
result = result.drop(['month','day','time_cos','day_num','time_sin','day_cos','day_sin',
                      'max_temp','min_temp','avg_wind',
                      'year_num','year_sin','year_cos','month_sin','month_cos'], axis=1)
result.head()

,latitude,longitude,time_num,pred_pickups,pickups
0,40.806656,-73.953781,0.802083,1.629808,NaN
1,40.758591,-73.937302,0.802083,1.201256,NaN
2,40.831375,-73.857651,0.802083,1.041130,NaN
3,40.797043,-73.938675,0.802083,1.201256,NaN
4,40.832748,-73.861771,0.802083,1.041130,NaN


In [44]:
# Exclude all records that have less than 2 pickups or less than 1.5 predicted pickups.
# This is to reduce the size of the resulting dataset in Tableau
result = result[(result['pred_pickups'] >= 1.5) | (result['pickups'] >= 1.5)]
print result.shape
result.head()

(5444, 5)


,latitude,longitude,time_num,pred_pickups,pickups
0,40.806656,-73.953781,0.802083,1.629808,NaN
7,40.803909,-73.955154,0.802083,1.629808,NaN
9,40.840988,-73.940048,0.802083,1.649880,NaN
20,40.803909,-73.963394,0.802083,1.629808,NaN
27,40.806656,-73.942795,0.802083,1.629808,NaN


In [45]:
# Write to csv
result.to_csv('./data/predictions-2013-11-13-weather.csv')

In [46]:
# Sanity check
result.describe()

,latitude,longitude,time_num,pred_pickups,pickups
count,5444.000000,5444.000000,5444.000000,5444.000000,0
mean,40.782597,-73.890616,0.580968,1.812473,NaN
std,0.033577,0.051139,0.279778,0.586620,NaN
min,40.713272,-74.012833,0.218750,1.629808,NaN
25%,40.758591,-73.940048,0.281250,1.629808,NaN
50%,40.803909,-73.901596,0.614583,1.629808,NaN
75%,40.808029,-73.838425,0.885417,1.788378,NaN
max,40.840988,-73.830185,0.968750,4.646723,NaN


### The result

In [47]:
%%html
<script type='text/javascript' src='https://public.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 804px; height: 569px;'><noscript><a href='#'><img alt='Pickups in NYC on May 1, 2015Play around with the slider below to see pickup densities at different times of the day ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;KD&#47;KD45Y8HBN&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz' width='804' height='569' style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='path' value='shared&#47;KD45Y8HBN' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;KD&#47;KD45Y8HBN&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='showVizHome' value='no' /><param name='showTabs' value='y' /><param name='bootstrapWhenNotified' value='true' /></object></div>